## Question 1: **Answer**

In [1]:
import pandas as pd
import sqlite3

In [ ]:
def add_data_to_database(input_data_1, input_data_2, input_data_3):
    try:
        # Creating DataFrame
        df_1 = pd.read_csv(input_data_1,  encoding='latin1')
        df_2 = pd.read_csv(input_data_2,  encoding='latin1')
        df_3 = pd.read_csv(input_data_3,  encoding='latin1')
        # Connecting to database
        engine = sqlite3.connect("champions_league.db")
        df_1.to_sql('players', con=engine, index = False)
        df_2.to_sql('goals', con=engine, index = False)
        df_3.to_sql('matches', con=engine, index = False)
        engine.close()
    except Exception as e:
            print(f'Error: {e}')

# Loading data
input_data_1 = "/content/players.csv"
input_data_2 = "/content/goals.csv"
input_data_3 = "/content/matches.csv"
add_data_to_database(input_data_1,input_data_2, input_data_3)

In [3]:
# Loading SQL Magic extension
%load_ext sql

In [4]:
# Connecting to the database
%sql sqlite:///champions_league.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM players
LIMIT 5;

 * sqlite:///champions_league.db
Done.


PLAYER_ID,FIRST_NAME,LAST_NAME,NATIONALITY,DOB,TEAM,JERSEY_NUMBER,POSITION,HEIGHT,WEIGHT,FOOT
ply510,None,Danilo,Brazil,7 15 1991,Juventus,6.0,Defender,184.0,78.0,R
ply511,Mattia De,Sciglio,Italy,10 20 1992,Juventus,2.0,Defender,182.0,78.0,R
ply512,Federico,Gatti,Italy,6 24 1998,Juventus,15.0,Defender,190.0,84.0,R
ply513,Daniele,Rugani,Italy,7 29 1994,Juventus,24.0,Defender,190.0,84.0,R
ply514,Enzo,Barrenechea,Argentina,5 22 2001,Juventus,45.0,Midfielder,186.0,81.0,L


In [7]:
%%sql
SELECT * FROM goals
LIMIT 5;

 * sqlite:///champions_league.db
Done.


GOAL_ID,MATCH_ID,PID,DURATION,ASSIST,GOAL_DESC
gl470,mt154,ply1479,4,ply864,right-footed shot
gl471,mt154,ply1075,28,ply864,penalty
gl472,mt154,ply585,69,ply864,right-footed shot
gl473,mt155,ply396,12,ply653,right-footed shot
gl474,mt155,ply1064,81,ply653,right-footed shot


In [8]:
%%sql
SELECT * FROM matches
LIMIT 5;

 * sqlite:///champions_league.db
Done.


MATCH_ID,SEASON,DATE_TIME,HOME_TEAM,AWAY_TEAM,STADIUM,HOME_TEAM_SCORE,AWAY_TEAM_SCORE,PENALTY_SHOOT_OUT,ATTENDANCE
mt1,2021-2022,15-SEP-21 08.00.00.000000000 PM,Manchester City,RB Leipzig,Etihad Stadium,6,3,0,38062
mt2,2021-2022,15-SEP-21 08.00.00.000000000 PM,Club Brugge KV,Paris Saint-Germain,Jan Breydel Stadion,1,1,0,27546
mt3,2021-2022,28-SEP-21 08.00.00.000000000 PM,Paris Saint-Germain,Manchester City,Parc des Princes,2,0,0,37350
mt4,2021-2022,28-SEP-21 08.00.00.000000000 PM,RB Leipzig,Club Brugge KV,Red Bull Arena,1,2,0,23500
mt5,2021-2022,19-OCT-21 05.45.00.000000000 PM,Club Brugge KV,Manchester City,Jan Breydel Stadion,1,5,0,24915


1. Return the player name, position, team, and the number of goals they contributed to (goals scored or assisted).

## Question 2: **Answer**

In [9]:
%%sql
SELECT
    g.GOAL_DESC,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM goals WHERE GOAL_DESC
                                                         IS NOT NULL ), 2)
    AS percentage
FROM goals g
WHERE g.GOAL_DESC IN ('left-footed shot', 'right-footed shot', 'header')
GROUP BY g.GOAL_DESC;

 * sqlite:///champions_league.db
Done.


GOAL_DESC,percentage
header,14.27
left-footed shot,27.93
right-footed shot,42.24


## Question 3: **Answer**

In [10]:
%%sql
-- Create index to optimize JOIN and filtering operations
CREATE INDEX IF NOT EXISTS idx_goals_pid_duration ON goals(PID);
CREATE INDEX IF NOT EXISTS idx_players_team_id ON players(TEAM, PLAYER_ID);

 * sqlite:///champions_league.db
Done.
Done.


[]

In [11]:
%%sql
-- Aggregating total goals and goals per each half
WITH goal_counts AS (
    SELECT
        p.TEAM,
        p.PLAYER_ID,
        p.FIRST_NAME || ' ' || p.LAST_NAME AS player_name,
        COUNT(g.GOAL_ID) AS total_goals,
        SUM(CASE WHEN g.DURATION <= 45 THEN 1 ELSE 0 END) AS first_half_goals,
        SUM(CASE WHEN g.DURATION > 45 THEN 1 ELSE 0 END) AS second_half_goals
    FROM players p
    JOIN goals g
    ON p.PLAYER_ID = g.PID
    GROUP BY p.TEAM, p.PLAYER_ID, player_name
),
--Ranking players by the number of goals scored
rank_goal_scorers AS (
    SELECT
        TEAM,
        PLAYER_ID,
        player_name,
        total_goals,
        first_half_goals,
        second_half_goals,
        ROW_NUMBER() OVER (PARTITION BY TEAM ORDER BY total_goals DESC) AS rank
    FROM goal_counts
)
-- main query to return overall top goal scorer
SELECT
    TEAM,
    PLAYER_ID,
    player_name,
    total_goals,
    first_half_goals,
    second_half_goals
FROM rank_goal_scorers
WHERE rank = 1
  AND TEAM IS NOT NULL
  AND total_goals = (SELECT MAX(total_goals) FROM goal_counts);

 * sqlite:///champions_league.db
Done.


TEAM,PLAYER_ID,player_name,total_goals,first_half_goals,second_half_goals
FC Barcelona,ply398,Robert Lewandowski,54,23,31


## Question 4: **Answer**

In [12]:
%%sql
-- filtering for header and forward
WITH forward_header_goals AS (
    SELECT
        p.PLAYER_ID,
        p.HEIGHT,
        g.GOAL_ID
    FROM players p
    LEFT JOIN goals g
    ON p.PLAYER_ID = g.PID
    WHERE p.POSITION LIKE '%Forward%' AND g.GOAL_DESC LIKE '%header%'
),
-- filtering for height categories
height_categories AS (
    SELECT
        PLAYER_ID,
        HEIGHT,
        CASE
            WHEN HEIGHT >= 180 THEN '180 and above'
            WHEN HEIGHT < 180 THEN 'Below 180'
        END AS height_category
    FROM forward_header_goals
),
-- Aggregating goals for the two categories
header_goal_counts AS (
    SELECT
        height_category,
        COUNT(*) AS header_goals
    FROM height_categories
    WHERE height_category IN ('180 and above', 'Below 180')
    GROUP BY height_category
),
-- Aggregating total forwards
total_forwards AS (
    SELECT COUNT(*) AS total_forwards
    FROM players
    WHERE POSITION LIKE '%Forward%'
)
-- Returning the goals for the two categories and percentages
SELECT
    gc.height_category,
    gc.header_goals,
    ROUND((gc.header_goals * 100.0) / tf.total_forwards, 2) AS percentage_of_total
FROM header_goal_counts gc
 -- This join is needed to combine the CTEs
JOIN total_forwards tf ON 1 = 1
ORDER BY gc.header_goals DESC;

 * sqlite:///champions_league.db
Done.


height_category,header_goals,percentage_of_total
180 and above,129,20.31
Below 180,44,6.93


## Question 5: **Answer**

In [13]:
%%sql
UPDATE matches
SET DATE_TIME = DATE('20'|| SUBSTR(DATE_TIME, 8, 2) || '-' ||
                    CASE SUBSTR(DATE_TIME, 4, 3)
                        WHEN 'JAN' THEN '01'
                        WHEN 'FEB' THEN '02'
                        WHEN 'MAR' THEN '03'
                        WHEN 'APR' THEN '04'
                        WHEN 'MAY' THEN '05'
                        WHEN 'JUN' THEN '06'
                        WHEN 'JUL' THEN '07'
                        WHEN 'AUG' THEN '08'
                        WHEN 'SEP' THEN '09'
                        WHEN 'OCT' THEN '10'
                        WHEN 'NOV' THEN '11'
                        WHEN 'DEC' THEN '12'
                    END || '-' || SUBSTR(DATE_TIME, 1, 2));

 * sqlite:///champions_league.db
744 rows affected.


[]

In [14]:
%%sql
SELECT
    COUNT(*) AS count_of_none_values
FROM matches
WHERE DATE_TIME IS NULL;

 * sqlite:///champions_league.db
Done.


count_of_none_values
0


In [15]:
%%sql
UPDATE players
SET DOB = DATE(REPLACE(REPLACE((CASE
                    WHEN DOB LIKE '_ __ ____' THEN SUBSTR(TRIM(DOB), 6, 4)||' -0'||SUBSTR(DOB, 1, 2)||'-'||SUBSTR(DOB, 3, 2)
                    WHEN DOB LIKE '__ __ ____' THEN SUBSTR(DOB, 7, 4)||'-'||SUBSTR(DOB, 1, 2)||'-'||SUBSTR(DOB, 4, 2)
                    WHEN DOB LIKE '__ _ ____' THEN SUBSTR(DOB, 6, 4)||'-'||SUBSTR(DOB, 1, 2)||'-0'||SUBSTR(DOB, 4, 2)
                    WHEN DOB LIKE '_ _ ____' THEN SUBSTR(DOB, 5, 4)||' -0'||SUBSTR(DOB, 1, 2)||'-0'||SUBSTR(DOB, 3, 2)
                    ELSE DOB -- Fallback for unmatched formats
                END),' -', '-'),' -0', '-0' ))
WHERE DOB IS NOT NULL;

 * sqlite:///champions_league.db
2755 rows affected.


[]

In [16]:
%%sql
DELETE FROM players
WHERE DOB IS NULL;

 * sqlite:///champions_league.db
14 rows affected.


[]

In [17]:
%%sql
SELECT
    COUNT(*) AS count_of_none_values
FROM players
WHERE DOB IS NULL;

 * sqlite:///champions_league.db
Done.


count_of_none_values
0


In [18]:
%%sql
WITH player_ages AS (
    SELECT
        g.GOAL_ID,
        p.PLAYER_ID,
        ROUND((JULIANDAY(m.DATE_TIME) - JULIANDAY(p.DOB)) / 365, 2)
        AS age_at_goal
    FROM goals g
    JOIN players p
    ON  g.PID = p.PLAYER_ID
    JOIN matches m
    ON  g.MATCH_ID = m.MATCH_ID
    WHERE m.SEASON = '2021-2022'
),
-- Aggregating goals per age group
age_range_goals AS (
    SELECT
        CASE
            WHEN age_at_goal BETWEEN 26 AND 30 THEN '26-30'
            WHEN age_at_goal BETWEEN 31 AND 35 THEN '31-35'
        END AS age_range,
        COUNT(GOAL_ID) AS total_goals
    FROM  player_ages
    WHERE age_at_goal BETWEEN 26 AND 30 OR
          age_at_goal  BETWEEN 31 AND 35
    GROUP BY age_range
)
-- Returning the results
SELECT
    age_range,
    total_goals
FROM age_range_goals;

 * sqlite:///champions_league.db
Done.


age_range,total_goals
26-30,112
31-35,73


In [19]:
# %%sql
# DROP TABLE matches;

# DROP TABLE players;

# DROP TABLE goals;